In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt

Configure the Device for gpu or cpu

In [2]:
#Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cuda


Setup Hyper parameters for the model

In [30]:
#Hyperparameters
num_epochs = 5 # amount of times the model will see the data
batch_size = 10 # amount of data to be processed at once
learning_rate = 0.001 # gradient descent step size

Load the data

In [31]:
#Transform image Data
# transforms the data into a tensor and normilizes it
# transform = transforms.Compose(
#             [transforms.ToTensor(),
#             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 
''' normalizes the data to be between -1 and 1 
    Color images have three channels (red, green, blue),therefore you need three parameters to normalize
    each channel.The first tuple (0.5, 0.5, 0.5) is the mean for all three channels 
    and the second (0.5, 0.5, 0.5) is the standard deviation for all three channels.
    if data value is 0 then when normalized we have:
    (0 - 0.5(mean)) / 0.5(std) = -1
    if data value is 1 then when normalized we have:
    (1 - 0.5) / 0.5 = 1
'''
# We could also turn the data into grayscale images by using:
# this will turn the values between 0 and 1
# 0 = black and 1 = white and in between is a shade of gray
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Grayscale(num_output_channels=1)]) 

In [41]:
#Load Data
train_dataset = torchvision.datasets.ImageFolder(root='emotion_dataset/train', transform=transform)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [33]:
train_data_loader = iter(train_data_loader)
print(next(train_data_loader)[0].shape)
# first number in the shape is the batch size
# second number in the shape is the number of channels
# third number in the shape is the height of the image
# fourth number in the shape is the width of the image

torch.Size([10, 1, 48, 48])


In [45]:
for data in train_data_loader:
    print(data[0][0])
    break
# the train_data_loader has 2 elements, the first element is the image data and the second element is the labels
# the image data is a tensor of shape (batch_size, channels, height, width)
# the labels are a tensor of shape (batch_size)
# if we do data[0][0] we are getting the first image in the batch
# if we do data[0] we are getting the whole batch of images
# if we do data[1] we are getting the whole batch of labels

tensor([[[0.3059, 0.2510, 0.2549,  ..., 0.8470, 0.8038, 0.8038],
         [0.2353, 0.2549, 0.3725,  ..., 0.8234, 0.8391, 0.8274],
         [0.2274, 0.2157, 0.2196,  ..., 0.8274, 0.8313, 0.8548],
         ...,
         [0.1216, 0.1333, 0.1647,  ..., 0.1961, 0.2510, 0.2823],
         [0.1216, 0.1451, 0.1647,  ..., 0.2274, 0.2823, 0.2902],
         [0.1490, 0.1490, 0.1686,  ..., 0.2431, 0.2862, 0.2666]]])
